### LOADING DEPENDENCIES

In [ ]:
!pip install python-docx
import re
import glob
from docx import Document 
import unicodedata
import spacy
nlp = spacy.load('en_core_web_sm')
from spacy import displacy

In [ ]:
# https://discuss.huggingface.co/t/error-with-new-tokenizers-urgent/2847/5
!pip install --no-cache-dir transformers sentencepiece

### LOADING DATA:

In [ ]:
# We pushed the SOW documents on github and clone the github repo whenever we need the documents
#This is faster than mounting our colab notebook to google drive
!git clone https://github.com/NLP-Contracts/NLP-summarization.git

fatal: destination path 'NLP-summarization' already exists and is not an empty directory.


In [ ]:
%cd NLP-summarization/Sample\ SoW\ docs

/content/NLP-summarization/Sample SoW docs


In [ ]:
list_docsNames = glob.glob('*.docx') #USING GLOB TO GET NAMES OF PROJECTS
 
docs = []
st = ""
for docsName in list_docsNames:
  docs.append(st.join([p.text for p in Document(docsName).paragraphs])) #USING DOCUMENT TO LOAD PROJECTS

### DATA CLEANING:

In [ ]:
#GENERIC CLEANING METHODS
# The basic_cleaner function will be applied first in the main corpus 
#at which we will select the sections using regex pattern matching
def basic_cleaner(s):
  s = s.lower()
  s = re.sub(r'\n', '', s)
  s = re.sub(r'\t', '', s)
  s = re.sub(r' + ', '', s)
  return(s)
#after select each section using regex pattern matching
#we apply the extra_cleaner function to remove punctuations 
#excpet "." & ",", in addition the section numbers and information inside brackets 
# will be removed as well
def extra_cleaner(s):
  # s= re.sub(',', ' ', s)
  s= re.sub('/', '', s)
  s= re.sub(':', '', s)
  s= re.sub('\'', '', s)
  s= re.sub('-', '', s)
  s= re.sub('/', '', s)
  s= re.sub('<', '', s)
  s= re.sub('>', '', s)
  s=re.sub(r'\d{1,2}\.\d', '', s) # remove subsection numbers by removing digit numbers (that have)\d{1,2}\.\d pattern
  s=re.sub(r'\([^)]*\)', '', s) #remove brakets and anything thats inside the brakets
  return(s)

################### need to implement capatization: 
def sum_format_cleaner(sum): #To fix the output of each section. Captalizing and adding "." at the end of each section.
  return(' '.join(map(str,sum)))
 
c_docs = [basic_cleaner(_) for _ in docs]


### Section Selection

In [ ]:
# It has to be noted that often the section names are not consistant in all documents that is why we applied an "or", "|" symbol 
# to find either or matched of a string.For instance the "section9_charges" section has 
#"charges, expenses and payment terms" and in some documents"fees, expenses and payment terms" or "expenses and payment terms"
#which represnet the same section 
section3_services=[' '.join(map(str, (re.findall('(?:.0services?|.sevices?| services 3.1?)(.*?) (?:term and schedule?|and schedule?|term and?)', i)))) for i in c_docs]
section4_schedule=[' '.join(map(str, (re.findall('(?:term and schedule?|and schedule?|term and?)(.*?)(?:place of performance?|place of performance and hours?|performance and hours)', i)))) for i in c_docs]
section5_PPH = [' '.join(map(str, (re.findall('(?:place of performance and hours?|performance and hours?)(.*?)(?:structure and roles|and roles?)', i)))) for i in c_docs]
section6_roles = [' '.join(map(str, (re.findall('(?:structure and roles ?|and roles?)(.*?)(?:general responsibilities|responsibilities?)', i)))) for i in c_docs]
section7_responsibilities = [' '.join(map(str, (re.findall('(?:general responsibilities)(.*?)(?:charges, expenses and payment terms|fees, expenses and payment terms?|expenses and payment terms?|milestones, deliverables, and acceptance criteria?|8.0 intentionally left blank?|.0 intentionally left blank)', i)))) for i in c_docs]
section9_charges = [' '.join(map(str, (re.findall('(?:charges, expenses and payment terms|fees, expenses and payment terms?|expenses and payment terms?)(.*?)(?:specific service levels)', i)))) for i in c_docs]
section12_assumptions=[' '.join(map(str, (re.findall('(?:assumptions and additional provisions?)(.*?)(?:addresses for administration and invoicing)', i)))) for i in c_docs]
section14_agreement = [' '.join(map(str, (re.findall('(?:.0 agreement?)(.*?)(?:agreed and accepted?)', i)))) for i in c_docs]

#Applyinf the "extra_cleaner" function to take out brackets, subsection numbers and most of the punctuations
c_section3_services = [extra_cleaner(i) for i in section3_services]
c_section4_schedule = [extra_cleaner(i) for i in section4_schedule]
c_section5_PPH = [extra_cleaner(i) for i in section5_PPH]
c_section6_roles = [extra_cleaner(i) for i in section6_roles]
c_section7_responsibilities = [extra_cleaner(i) for i in section7_responsibilities]
c_section9_charges = [extra_cleaner(i) for i in section9_charges]
c_section12_assumptions = [extra_cleaner(i) for i in section12_assumptions]
c_section14_agreement = [extra_cleaner(i) for i in section14_agreement]
# This corpus is the collection of sections which we selectively chose to remain in the document
corpus = [] 
for i in range(len(docs)): # range of the loop is the number of documents that are introduced 
  corpus.append(c_section3_services[i] + "\n"+ c_section4_schedule[i] + "\n" + c_section5_PPH[i] + "\n" + c_section6_roles[i] + "\n" + c_section7_responsibilities[i] + "\n" + c_section9_charges[i] + "\n" + c_section12_assumptions[i] + "\n" + c_section14_agreement[i])

# We are using Corpus[0]  
AKA--> TI_SOW_58_2019_TM_MITS_Stratus_mock.docx as our sample document to display our summarization results
It must be noted that we tested our model for all documents but for easy representation of our result we just decided to show one sample corpus to Mahmadul. 

###BART Model:
The Bart model was proposed in BART: Denoising Sequence-to-Sequence Pre-training for Natural Language Generation, Translation, and Comprehension by Mike Lewis, Yinhan Liu, Naman Goyal, Marjan Ghazvininejad, Abdelrahman Mohamed, Omer Levy, Ves Stoyanov and Luke Zettlemoyer on 29 Oct, 2019.

According to the abstract:

* Bart uses a standard seq2seq/machine translation architecture with a bidirectional encoder (like BERT) and a left-to-right decoder (like GPT).
* The pretraining task involves randomly shuffling the order of the original sentences and a novel in-filling scheme, where spans of text are replaced with a single mask token.
* BART is particularly effective when fine tuned for text generation but also works well for comprehension tasks. It matches the performance of RoBERTa with comparable training resources on GLUE and SQuAD, achieves new state-of-the-art results on a range of abstractive dialogue, question answering, and summarization tasks, with gains of up to 6 ROUGE.


https://huggingface.co/transformers/model_doc/bart.html

https://github.com/pytorch/fairseq/tree/master/examples/bart




In [ ]:
from transformers import BartTokenizer, BartForConditionalGeneration, BartConfig 

model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn') #Loading Bart facebook/bart-large-cnn pretrained model
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')


###FULL DOCUMENT SUMMARIZATION USING BART:

In [ ]:
def bart_full_summ(doc_num):
  #Full Corpus:
  ARTICLE_TO_SUMMARIZE = corpus[doc_num]
  inputs = tokenizer([ARTICLE_TO_SUMMARIZE], max_length=1024, return_tensors='pt', truncation=True)
  # Generate Summary
  summary_ids = model.generate(inputs['input_ids'], num_beams=4, early_stopping=True)
  corpus_sum = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids]
  corpus_sum = sum_format_cleaner(corpus_sum)
  return(corpus_sum)


def summaize_full_sec_doc(n_doc):
  
  sum_full_sec= bart_full_summ(n_doc)
    
  if nlp(sum_full_sec).ents:
    displacy.render(nlp(sum_full_sec), style="ent",jupyter=True) # shows the Named Entity Recognition labels as highlights if applicable to that summary
                                                        # This will assist the reader while looking at the summarized document
  else:
    display(sum_full_sec)                                 #If The section doesnt have NER labels then display the summarized section as is 
  print("\033[95m" + "Overal total words from the sectioned document After Summarization:"+ "\033[0m",(len(sum_full_sec.split())))
  print("\033[95m" + "Overal total words from the sectioned document before Summarization:"+ "\033[0m",(len(corpus[n_doc].split())))
  print("\033[95m" + "Ratio to the Original document: %"+ "\033[0m",(len(sum_full_sec.split())/(len(corpus[n_doc].split()))*100))
  print("\033[95m" + "Overal Orignal document words before Summarization:"+ "\033[0m",(len(c_docs[n_doc].split())))
  print("\033[95m" + "Ratio to the Original document: %"+ "\033[0m",(len(sum_full_sec.split())/(len(c_docs[n_doc].split()))*100))


#Result/ Excecution of the bart_full_summ Summarization function

This result shows that the Bart model doesnt retain valuable information such as starting and ending dates of the contract as well as the charges (CAD) related to the contract with only 57 words. That is why we approached this problem by summarizing the document section by section to retain the valuable information for whomever reading the summarized document.

In [ ]:
summaize_full_sec_doc(0)

Overal total words from the sectioned document After Summarization: 57
Overal total words from the sectioned document before Summarization: 2452
Ratio to the Original document: % 2.32463295269168
Overal Orignal document words before Summarization: 4246
Ratio to the Original document: % 1.3424399434762129


###SECTION BY SECTION SUMMARIZATION USING BART:

In [ ]:
def bart_section_sum(doc_num):     #Taking the Doc number that we want to summarize 
          
  #Summarizing each section indivdually. 

  #Section3:
  ARTICLE_TO_SUMMARIZE = c_section3_services[doc_num]
  inputs = tokenizer([ARTICLE_TO_SUMMARIZE], max_length=1024, return_tensors='pt')
  # Generate Summary
  summary_ids = model.generate(inputs['input_ids'], num_beams=4, early_stopping=True)
  service_sum = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids]
  service_sum = sum_format_cleaner(service_sum)


  #Section4:
  ARTICLE_TO_SUMMARIZE = c_section4_schedule[doc_num]
  inputs = tokenizer([ARTICLE_TO_SUMMARIZE], max_length=1024, return_tensors='pt')
  # Generate Summary
  summary_ids = model.generate(inputs['input_ids'], num_beams=4, early_stopping=True)
  schedule_sum = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids]
  schedule_sum = sum_format_cleaner(schedule_sum)


  #Section5:
  ARTICLE_TO_SUMMARIZE = c_section5_PPH[doc_num]
  inputs = tokenizer([ARTICLE_TO_SUMMARIZE], max_length=1024, return_tensors='pt')
  # Generate Summary
  summary_ids = model.generate(inputs['input_ids'], num_beams=4, early_stopping=True)
  PPH_sum = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids]
  PPH_sum = sum_format_cleaner(PPH_sum)


  #Section6:
  ARTICLE_TO_SUMMARIZE = c_section6_roles[doc_num]
  inputs = tokenizer([ARTICLE_TO_SUMMARIZE], max_length=1024, return_tensors='pt')
  # Generate Summary
  summary_ids = model.generate(inputs['input_ids'], num_beams=4, early_stopping=True)
  role_sum = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids]
  role_sum = sum_format_cleaner(role_sum)

  #Section7:
  ARTICLE_TO_SUMMARIZE = c_section7_responsibilities[doc_num]
  inputs = tokenizer([ARTICLE_TO_SUMMARIZE], max_length=1024, return_tensors='pt')
  # Generate Summary
  summary_ids = model.generate(inputs['input_ids'], num_beams=4, early_stopping=True)
  resp_sum = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids]
  resp_sum = sum_format_cleaner(resp_sum)

  #Section9:
  ARTICLE_TO_SUMMARIZE = c_section9_charges[doc_num]
  inputs = tokenizer([ARTICLE_TO_SUMMARIZE], max_length=1024, return_tensors='pt')
  # Generate Summary
  summary_ids = model.generate(inputs['input_ids'], num_beams=4, early_stopping=True)
  charge_sum = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids]
  charge_sum = sum_format_cleaner(charge_sum)


  #Section12:
  ARTICLE_TO_SUMMARIZE = c_section12_assumptions[doc_num]
  inputs = tokenizer([ARTICLE_TO_SUMMARIZE], max_length=1024, return_tensors='pt')
  # Generate Summary
  summary_ids = model.generate(inputs['input_ids'], num_beams=4, early_stopping=True)
  assum_sum = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids]
  assum_sum = sum_format_cleaner(assum_sum)

  #Section14:
  ARTICLE_TO_SUMMARIZE = c_section14_agreement[doc_num]
  inputs = tokenizer([ARTICLE_TO_SUMMARIZE], max_length=1024, return_tensors='pt')
  # Generate Summary
  summary_ids = model.generate(inputs['input_ids'], num_beams=4, early_stopping=True)
  agree_sum = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids]
  agree_sum = sum_format_cleaner(agree_sum)

  return service_sum, schedule_sum, PPH_sum , role_sum, resp_sum, charge_sum, assum_sum, agree_sum


In [ ]:
def summaize_by_section(n_doc):
  
  service_sum,schedule_sum,PPH_sum,role_sum,resp_sum, charge_sum,assum_sum,agree_sum= bart_section_sum(n_doc) #This function t5_section(n_doc) 
                                                                                                        #will select the document number 
                                                                                                        #which we want to summarize and spits out seven
                                                                                                        #summarized sections which will be displayed in the 
                                                                                                        #for loop below
  original_sections=[c_section3_services,c_section4_schedule,c_section5_PPH,c_section6_roles,c_section7_responsibilities,c_section9_charges,c_section12_assumptions,c_section14_agreement]
  sum_sections={"Services":service_sum,
            "Schedule":schedule_sum,
            "Place of Performance and Hours":PPH_sum,
            "Role":role_sum,
            "Responsibilities":resp_sum,
            "Charge":charge_sum,
            "Assumptions":assum_sum,
            "Agreement":agree_sum}

  for v,i in zip(sum_sections.items(),original_sections): #I zipped sum_sections and original_sections so to display 
                                                          #the length of words in the original_sections before summarization and after summarization

    print("\033[95m" + v[0]+ "\033[0m") #sum_sections is a dictionary which in a zip will be transformed to a list of lists 
                                        #where the list with index 0 becomes the keys and 
                                        #list with index 1 becomes the summarized strings
                                        #This print will display the section names
    print("\t")


    if nlp(v[1]).ents:
     displacy.render(nlp(v[1]), style="ent",jupyter=True) # shows the Named Entity Recognition labels as highlights if applicable to that summary
                                                          # This will assist the reader while looking at the summarized document
    else:
      display(v[1])                                       #If The section doesnt have NER labels then display the summarized section as is 
    
    print("\t")
    print("\033[33m" + "summarized lenght of section"+ "\033[0m",len(v[1].split())) # displays the word length of the sum_sections 
    print("\033[33m" + "original lenght of section"+ "\033[0m",len(i[n_doc].split())) # displays the word length of the original_sections 
    
#this for loop below helps in calculating the overal word lenght of the full summarized document with all sections 
  sum=0
  h=[]
  for k,v in sum_sections.items():
    h.append(len(v.split()))
  for i in h:
    sum=i+sum
  print("\t")
  print("\033[95m" + "Overal total words from the sectioned document in the Summarized Version:"+ "\033[0m",sum)
  print("\033[95m" + "Overal total words from the sectioned document before Summarization:"+ "\033[0m",(len(corpus[n_doc].split())))
  print("\033[95m" + "Ratio to the sectioned document: %"+ "\033[0m",(sum/(len(corpus[n_doc].split())))*100)
  print("\033[95m" + "Overal Orignal document words before Summarization:"+ "\033[0m",(len(c_docs[n_doc].split())))
  print("\033[95m" + "Ratio to the Original document: %"+ "\033[0m",(sum/(len(c_docs[n_doc].split())))*100)

    

In [ ]:
summaize_by_section(0)

Services
	


'subject to the agreement, the sowspecific scope of services shall include the following. this sow provides a broad set of it services that are all delivered in a time and materials and staff augmentation delivery model. the resources provided by ti service representatives are directed and managed by telus manager and their scope of duties is therefore open to change.'

	
summarized lenght of section 60
original lenght of section 122
Schedule
	


	
summarized lenght of section 72
original lenght of section 147
Place of Performance and Hours
	


	
summarized lenght of section 68
original lenght of section 192
Role
	


...

	
summarized lenght of section 69
original lenght of section 383
Charge
	


	
summarized lenght of section 61
original lenght of section 585
Assumptions
	


	
summarized lenght of section 46
original lenght of section 577
Agreement
	


'the parties acknowledge and agree that the terms and conditions of the agreement shall govern this statement of work. this sow and any change orders issued hereunder may be executed by the exchange of signed counterparts by facsimile transmission or electronically in pdf or similar secure format. when taken together will constitute one and the same document.'

	
summarized lenght of section 57
original lenght of section 71
	
Overal total words from the sectioned document in the Summarized Version: 484
Overal total words from the sectioned document before Summarization: 2452
Ratio to the sectioned document: % 19.73898858075041
Overal Orignal document words before Summarization: 4246
Ratio to the Original document: % 11.398963730569948
